### 행안부 도로명주소 API 활용

In [12]:
import requests
from urllib.parse import quote # 주소창에 한글을 입력할 때 변환하는 메서드

- API Key 가져오기

In [13]:
from google.colab import files # 로컬에 있는 파일 업로드해서 가져옴. 그냥 드로그 앤 드롭해도 될듯?
up = files.upload()

Saving roadapikey.txt to roadapikey.txt


In [14]:
with open('roadapikey.txt') as rf:
    road_key = rf.read()

- url 만들기

In [15]:
# 한글은 반드시 인코딩해야 한다.
quote('광주광역시청')

'%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C%EC%B2%AD'

In [16]:
# url은 copy-paste. 도로명주소 api에서 가져옴.
base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do' # POST GET에서 가져옴
params1 = f'currentPage=1&countPerPage=10&confmKey={road_key}' # 파라미터를 여러 개 붙일 때는 &를 사이에 둠
params2 = f'keyword={quote("광주광역시청")}&resultType=json'
url = f'{base_url}?{params1}&{params2}' # 기본 url 이후 파라미터를 넣을 때 ?를 붙임.

- 요청해서 결과 얻기

In [17]:
result = requests.get(url) #만들어진 url로 정보 가져옴
result.status_code # 정상적인 경우 200

200

- 결과 해석하기: JSON --> Python 데이터 구조로 변환

In [18]:
import json # json 패키지를 통해 데이터 구조 변환
res = json.loads(result.text)
res

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '1',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '111 Naebang-ro, Seo-gu, Gwangju',
    'rn': '내방로',
    'emdNm': '치평동',
    'zipNo': '61945',
    'roadAddrPart2': '(치평동)',
    'emdNo': '04',
    'sggNm': '서구',
    'jibunAddr': '광주광역시 서구 치평동 1200 광주광역시청',
    'siNm': '광주광역시',
    'roadAddrPart1': '광주광역시 서구 내방로 111',
    'bdNm': '광주광역시청',
    'admCd': '2914012000',
    'udrtYn': '0',
    'lnbrMnnm': '1200',
    'roadAddr': '광주광역시 서구 내방로 111(치평동)',
    'lnbrSlno': '0',
    'buldMnnm': '111',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '291403160004',
    'mtYn': '0',
    'bdMgtSn': '2914012000112000000003487',
    'buldSlno': '0'}]}}

In [35]:
result.text # 딕셔너리 처럼 생긴 문자열

'{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"1","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"111 Naebang-ro, Seo-gu, Gwangju","rn":"내방로","emdNm":"치평동","zipNo":"61945","roadAddrPart2":"(치평동)","emdNo":"04","sggNm":"서구","jibunAddr":"광주광역시 서구 치평동 1200 광주광역시청","siNm":"광주광역시","roadAddrPart1":"광주광역시 서구 내방로 111","bdNm":"광주광역시청","admCd":"2914012000","udrtYn":"0","lnbrMnnm":"1200","roadAddr":"광주광역시 서구 내방로 111(치평동)","lnbrSlno":"0","buldMnnm":"111","bdKdcd":"0","liNm":"","rnMgtSn":"291403160004","mtYn":"0","bdMgtSn":"2914012000112000000003487","buldSlno":"0"}]}}'

In [28]:
name = res['results']['juso'][0]['bdNm'] # 딕셔너리와 리스트가 혼합되어 있으니 일단 열어서 하나하나 보며 위치 찾기
name

'광주광역시청'

In [36]:
road_addr = res['results']['juso'][0]['roadAddr']
road_addr

'광주광역시 서구 내방로 111(치평동)'

- 광주광역시 주요 행정기관 주소

In [37]:
places = '광주광역시청,광주광역시동구청,광주광역시북구청,광주광역시서구청,광주광역시서구청,광주광역시남구청,광주광역시광산구청'.split(',')
places

['광주광역시청',
 '광주광역시동구청',
 '광주광역시북구청',
 '광주광역시서구청',
 '광주광역시서구청',
 '광주광역시남구청',
 '광주광역시광산구청']

In [39]:
addr_dict = {}
for place in places:
    print(place)
    base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do' # POST GET에서 가져옴
    params1 = f'currentPage=1&countPerPage=10&confmKey={road_key}' # 파라미터를 여러 개 붙일 때는 &를 사이에 둠
    params2 = f'keyword={quote(place)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}' # 기본 url 이후 파라미터를 넣을 때 ?를 붙임.
    result = requests.get(url)
    res = json.loads(result.text)
    road_addr = res['results']['juso'][0]['roadAddr']
    addr_dict[place] = road_addr
#북구청에서 문제가 생김

광주광역시청
광주광역시동구청
광주광역시북구청


IndexError: ignored

In [40]:
base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do' # POST GET에서 가져옴
params1 = f'currentPage=1&countPerPage=10&confmKey={road_key}' # 파라미터를 여러 개 붙일 때는 &를 사이에 둠
params2 = f'keyword={quote("광주광역시 북구청")}&resultType=json'
url = f'{base_url}?{params1}&{params2}' # 기본 url 이후 파라미터를 넣을 때 ?를 붙임.
result = requests.get(url)
res = json.loads(result.text)
res
# 검색 자체가 안 됨.
# 광주광역시 북구청으로 하니 된다.(띄어쓰기 이슈)

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '0',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': []}}

In [43]:
places = '광주광역시청,광주광역시 동구청,광주광역시 북구청,광주광역시 서구청,광주광역시 남구청,광주광역시 광산구청'.split(',')
addr_list = []
for place in places:
    print(place)
    base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do' # POST GET에서 가져옴
    params1 = f'currentPage=1&countPerPage=10&confmKey={road_key}' # 파라미터를 여러 개 붙일 때는 &를 사이에 둠
    params2 = f'keyword={quote(place)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}' # 기본 url 이후 파라미터를 넣을 때 ?를 붙임.
    result = requests.get(url)
    res = json.loads(result.text)
    road_addr = res['results']['juso'][0]['roadAddr']
    addr_list.append(road_addr)
addr_list

광주광역시청
광주광역시 동구청
광주광역시 북구청
광주광역시 서구청
광주광역시 남구청
광주광역시 광산구청


['광주광역시 서구 내방로 111(치평동)',
 '광주광역시 동구 서남로 1(서석동)',
 '광주광역시 북구 우치로 77(용봉동)',
 '광주광역시 서구 경열로 33(농성동)',
 '광주광역시 남구 봉선로 1(봉선동)',
 '광주광역시 광산구 광산로29번길 15(송정동)']

- DataFrame으로 만들어 저장하기

In [44]:
import pandas as pd
df = pd.DataFrame({'장소':places, '도로명주소':addr_list})
df

,장소,도로명주소
0,광주광역시청,광주광역시 서구 내방로 111(치평동)
1,광주광역시 동구청,광주광역시 동구 서남로 1(서석동)
2,광주광역시 북구청,광주광역시 북구 우치로 77(용봉동)
3,광주광역시 서구청,광주광역시 서구 경열로 33(농성동)
4,광주광역시 남구청,광주광역시 남구 봉선로 1(봉선동)
5,광주광역시 광산구청,광주광역시 광산구 광산로29번길 15(송정동)


In [47]:
df.to_csv('광주광역시_주요_행정기관2.csv', index=False, encoding='euc-kr')